# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> Notebook for Chamber.jl

- Gihub: [Chamber.jl](https://github.com/brown-ccv/Chamber.jl)
- About Google Colaboratory: [FAQ](https://research.google.com/colaboratory/faq.html)

## Instructions (Google Colab only)
1. Change the Runtime to run Julia code: Click on _Runtime_ > _Change runtime type_. In the _Notebook settings_ window, select "Julia" from the _Runtime type_ dropdown menu. Click _Save_.

_Notes_:
* In a different enviroment (cluster or a local system), change the notebook's kernel to Julia.
* It may need ~20 minutes to install on Google Colab.

In [ ]:
using Pkg
Pkg.add("Chamber")

## Checking the Installation

In [ ]:
using Chamber
@doc chamber

## Try Chamber.jl

### Using Chamber

* Please use `float` data type for the values of `end_time`, `log_volume_km3`, `InitialConc_H2O`, `InitialConc_CO2`, `log_vfr` and `depth`.
* Prior to executing the `chamber` function, make sure to run `using Chamber`.
* Verify output files by clicking the folder icon on the left sidebar. The folder name will default to the timestamp if the `output_dirname` argument is not specified.

    <img src="images/instructions_3.png" height=200/> <img src="images/instructions_4.png" height=220/>

### Example 1: Silicic case

In [ ]:
composition = Silicic()
end_time = 3e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.04
InitialConc_CO2 = 0.001
log_vfr = -3.3
depth = 8e3
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)

### Example 2: Mafic case

In [ ]:
composition = Mafic()
end_time = 3e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.01
InitialConc_CO2 = 0.001
log_vfr = -3.3
depth = 8e3
output_dirname = "MyDirname"
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth, output_dirname)

### Example 3: Silicic saturated case

In [ ]:
composition = Silicic()
end_time = 3e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.05
InitialConc_CO2 = 0.0001
log_vfr = -3.3
depth = 8e3
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)

### Example 4: Mafic saturated case

In [ ]:
composition = Mafic()
end_time = 6e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.01
InitialConc_CO2 = 0.0008
log_vfr = -3.3
depth = 8e3
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)

## Save results to your Google Drive
To download the output files, mount your Google Drive by following these steps:

1. Change the runtime type to Python.

    <img src="images/instructions_5.png" height=320/>   <img src="images/instructions_6.png" height=250/>

2. Run the code cell below.
3. Afterward, you can drag the desired output files into the Drive folder.

    <img src="images/instructions_7.png" height=200/>


[Notice] Remember to change the runtime type back to Julia before running other Julia code cells.

In [ ]:
# runtime type: Python
from google.colab import drive
drive.mount('/content/drive')